In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time


# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# 문자열을 숫자로 변환
from sklearn.preprocessing import LabelEncoder
# 원핫인코딩
from tensorflow.keras.utils import to_categorical

# 학습데이터와 검증데이터로 나눈다
from sklearn.model_selection import train_test_split

# K폴드 교차검증
from sklearn.model_selection import KFold

# 저장된 모델을 복원
from tensorflow.keras.models import load_model

# epoch 마다 저장하기 위해 사용
from tensorflow.keras.callbacks import ModelCheckpoint

# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)


In [2]:
# 랜덤 시드 설정
np.random.seed(1)
tf.random.set_seed(1)

In [3]:
df1 = pd.read_csv('data/wine.csv', header=None)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [4]:
df1.info() # 문자열 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6497 non-null   float64
 1   1       6497 non-null   float64
 2   2       6497 non-null   float64
 3   3       6497 non-null   float64
 4   4       6497 non-null   float64
 5   5       6497 non-null   float64
 6   6       6497 non-null   float64
 7   7       6497 non-null   float64
 8   8       6497 non-null   float64
 9   9       6497 non-null   float64
 10  10      6497 non-null   float64
 11  11      6497 non-null   int64  
 12  12      6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [5]:
df1.isna().sum() # 결측치 확인

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
dtype: int64

In [6]:
# 입력과 결과로 나눈다.
X = df1.drop(12, axis=1)
y = df1[12]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: 12, Length: 6497, dtype: int64

In [7]:
# 데이터 표준화 
scaler1 = StandardScaler()
scaler1.fit(X)
X = scaler1.transform(X)
X

array([[ 0.14247327,  2.18883292, -2.19283252, ...,  0.19309677,
        -0.91546416, -0.93722961],
       [ 0.45103572,  3.28223494, -2.19283252, ...,  0.99957862,
        -0.58006813, -0.93722961],
       [ 0.45103572,  2.55330026, -1.91755268, ...,  0.79795816,
        -0.58006813, -0.93722961],
       ...,
       [-0.55179227, -0.6054167 , -0.88525328, ..., -0.47897144,
        -0.91546416,  0.20799905],
       [-1.32319841, -0.30169391, -0.12823371, ..., -1.016626  ,
         1.9354021 ,  1.35322771],
       [-0.93749534, -0.78765037,  0.42232597, ..., -1.41986693,
         1.09691202,  0.20799905]])

In [8]:
# 모델 설정
model = Sequential()

a1 = X.shape[1]
model.add(Dense(30, input_dim=a1))
model.add(Activation('relu'))

model.add(Dense(12))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))


In [9]:
# 학습 모델을 생성한다
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [10]:
# 모델들이 저장될 파일명
# epoch : 학습 횟수
# val_loss : 검증 데이터 손실률
path1 = 'model/17/{epoch}-{val_loss}.h5'
path2 = 'model/17/최종.h5'

# 저장 콜백 설정
# save_best_only : True로 설정하면 개선이 되었을 때만 저장한다.
call1 = ModelCheckpoint(filepath=path1, monitor='val_loss', 
                        save_best_only=True)
call2 = ModelCheckpoint(filepath=path2, monitor='val_loss',
                        save_best_only=True)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=1)


In [12]:
# 학습한다.

model.fit(X, y, epochs=200, batch_size=100, validation_data=[X_test,y_test], callbacks=[call1,call2])


Epoch 1/200
65/65 [==============================] - 1s 6ms/step - loss: 0.5226 - accuracy: 0.8387 - val_loss: 0.2880 - val_accuracy: 0.9577
Epoch 2/200
65/65 [==============================] - 0s 4ms/step - loss: 0.1507 - accuracy: 0.9769 - val_loss: 0.0787 - val_accuracy: 0.9815
Epoch 3/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0536 - accuracy: 0.9877 - val_loss: 0.0510 - val_accuracy: 0.9869
Epoch 4/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0371 - accuracy: 0.9906 - val_loss: 0.0423 - val_accuracy: 0.9892
Epoch 5/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0308 - accuracy: 0.9935 - val_loss: 0.0377 - val_accuracy: 0.9923
Epoch 6/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0273 - accuracy: 0.9943 - val_loss: 0.0348 - val_accuracy: 0.9931
Epoch 7/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0250 - accuracy: 0.9954 - val_loss: 0.0320 - val_accuracy: 0.9931
Epoch 8/200
6

Epoch 58/200
65/65 [==============================] - 0s 6ms/step - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.0086 - val_accuracy: 0.9969
Epoch 59/200
65/65 [==============================] - 0s 6ms/step - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.0085 - val_accuracy: 0.9977
Epoch 60/200
65/65 [==============================] - 0s 6ms/step - loss: 0.0064 - accuracy: 0.9986 - val_loss: 0.0081 - val_accuracy: 0.9977
Epoch 61/200
65/65 [==============================] - 0s 6ms/step - loss: 0.0062 - accuracy: 0.9986 - val_loss: 0.0079 - val_accuracy: 0.9977
Epoch 62/200
65/65 [==============================] - 0s 6ms/step - loss: 0.0061 - accuracy: 0.9988 - val_loss: 0.0078 - val_accuracy: 0.9977
Epoch 63/200
65/65 [==============================] - 0s 6ms/step - loss: 0.0059 - accuracy: 0.9988 - val_loss: 0.0073 - val_accuracy: 0.9977
Epoch 64/200
65/65 [==============================] - 0s 5ms/step - loss: 0.0057 - accuracy: 0.9988 - val_loss: 0.0074 - val_accuracy: 0.9977
Epoch 

65/65 [==============================] - 0s 4ms/step - loss: 9.5789e-04 - accuracy: 0.9995 - val_loss: 0.0017 - val_accuracy: 0.9985
Epoch 116/200
65/65 [==============================] - 0s 4ms/step - loss: 8.9136e-04 - accuracy: 0.9997 - val_loss: 0.0018 - val_accuracy: 0.9985
Epoch 117/200
65/65 [==============================] - 0s 4ms/step - loss: 9.2279e-04 - accuracy: 0.9994 - val_loss: 0.0018 - val_accuracy: 0.9985
Epoch 118/200
65/65 [==============================] - 0s 4ms/step - loss: 0.0012 - accuracy: 0.9995 - val_loss: 0.0019 - val_accuracy: 0.9985
Epoch 119/200
65/65 [==============================] - 0s 4ms/step - loss: 9.2038e-04 - accuracy: 0.9994 - val_loss: 0.0025 - val_accuracy: 0.9985
Epoch 120/200
65/65 [==============================] - 0s 4ms/step - loss: 8.5649e-04 - accuracy: 0.9997 - val_loss: 0.0017 - val_accuracy: 0.9992
Epoch 121/200
65/65 [==============================] - 0s 5ms/step - loss: 9.2751e-04 - accuracy: 0.9995 - val_loss: 0.0014 - val_accura

Epoch 171/200
65/65 [==============================] - 0s 4ms/step - loss: 9.4041e-04 - accuracy: 0.9997 - val_loss: 0.0019 - val_accuracy: 0.9985
Epoch 172/200
65/65 [==============================] - 0s 4ms/step - loss: 6.9644e-04 - accuracy: 0.9997 - val_loss: 0.0013 - val_accuracy: 0.9985
Epoch 173/200
65/65 [==============================] - 0s 4ms/step - loss: 7.2655e-04 - accuracy: 0.9995 - val_loss: 0.0015 - val_accuracy: 0.9985
Epoch 174/200
65/65 [==============================] - 0s 4ms/step - loss: 7.3579e-04 - accuracy: 0.9995 - val_loss: 0.0016 - val_accuracy: 0.9985
Epoch 175/200
65/65 [==============================] - 0s 4ms/step - loss: 6.7996e-04 - accuracy: 0.9995 - val_loss: 0.0015 - val_accuracy: 0.9985
Epoch 176/200
65/65 [==============================] - 0s 4ms/step - loss: 6.6054e-04 - accuracy: 0.9997 - val_loss: 0.0021 - val_accuracy: 0.9985
Epoch 177/200
65/65 [==============================] - 0s 4ms/step - loss: 6.6231e-04 - accuracy: 0.9995 - val_loss: 0

In [13]:
# 모델을 복원한다.
model2 = load_model(path2)
model2


In [14]:
a1 = model2.evaluate(X_test, y_test) # 저장한 파일이 맞는지 확인
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')


41/41 [==============================] - 0s 3ms/step - loss: 9.3061e-04 - accuracy: 0.9992
손실률 : 0.0009306095307692885
정확도 : 0.9992307424545288
